In [1]:
from enum import Enum, auto
from typing import List
from collections import Counter
import functools

In [2]:
with open("test_input7.txt") as fin:
    string_rows = [line.strip().split(" ") for line in fin]

for row in string_rows:
    print(row)

['32T3K', '765']
['T55J5', '684']
['KK677', '28']
['KTJJT', '220']
['QQQJA', '483']


This whole task is all about ordering and sorting. Therefore, we map our cards to integers - those are easier to sort. 

In [3]:
all_cards = ['2', '3', '4', '5', '6', '7', '8', '9', 'T', 'J', 'Q', 'K', 'A']
card_dict = dict(map(reversed, enumerate(all_cards)))
print(card_dict)

{'2': 0, '3': 1, '4': 2, '5': 3, '6': 4, '7': 5, '8': 6, '9': 7, 'T': 8, 'J': 9, 'Q': 10, 'K': 11, 'A': 12}


Now we create an overview of all possible and types. Note that overwrite `__lt__` (i.e. less than). This is the method used by python to determen wether a thing is smaller than another thing when ordering things. 

In [4]:
class HandValue(Enum):
    HIGH_CARD = auto()
    PAIR = auto()
    TWO_PAIR = auto()
    THREE_OF_A_KIND = auto()
    FULL_HOUSE = auto()
    FOUR_OF_A_KIND = auto()
    FIVE_OF_A_KIND = auto()
    
    def __lt__(self, b):
        return self.value < b.value


We can now compare hand types:

In [5]:
print(HandValue.TWO_PAIR < HandValue.THREE_OF_A_KIND)
print(HandValue.TWO_PAIR < HandValue.PAIR)

True
False


Now we need a structure for our different hands. Because we want to implement rather complex methods on these hands, we implement a class for it. This class has three member variables:

* `cards`: The string representation of the hand (e.g. `"2KJ2K"`)
* `value`: The type of hand that we hold (`HandValue`). This value is derived from the cards in hand.

I will explain the details below.

In [6]:
@functools.total_ordering
class Hand:
    def __init__(self, card_string: str):
        self.cards = card_string
        self.value = self._determine_hand_value()
    
    @staticmethod   
    def _map_card(c):
        return card_dict[c]
        
    def _count_cards(self):
        counter = Counter(self.cards)
        # We do not need the cards, just their multiplicities
        return counter
        
    def _determine_hand_value(self):
        # Count all cards
        numbers = list(self._count_cards().values())
        
        
        if 5 in numbers:
            return HandValue.FIVE_OF_A_KIND
        elif 4 in numbers:
            return HandValue.FOUR_OF_A_KIND
        elif 3 in numbers:
            # Is there another pair?
            if 2 in numbers: 
                return HandValue.FULL_HOUSE
            else:
                return HandValue.THREE_OF_A_KIND
        elif 2 in numbers:
            # Is there another pair?
            number_of_twos = numbers.count(2)
            if number_of_twos == 1:
                return HandValue.PAIR
            elif number_of_twos == 2:
                return HandValue.TWO_PAIR
            else:
                # This should never happen
                raise Exception("More than 5 cards detected. Cheater!")
        else:
            return HandValue.HIGH_CARD

    def __lt__(self, other):
        if self.value < other.value:
            return True
        elif other.value < self.value:
            return False
        else:
            for (l,r) in zip(map(self._map_card, self.cards), map(self._map_card, other.cards)):
                if l < r:
                    return True
                if r < l:
                    return False
            return False
        
    def __repr__(self):
        return f"{self.cards} ({self.value.name})"

The most important function in this class is `_determine_hand_value` that is used to derive the hand type from a given hand.
For this, we count the number of cards in our hand. Let us consider the example `"2KJ2K"`. First, we count the number of instances of cards in this string:

In [7]:
example_hand = Hand("2KJ2K")
example_hand._count_cards()

Counter({'2': 2, 'K': 2, 'J': 1})

So, we have two kings and twos each and a single Jack . We put this into a simple decision scheme: If there are 5 of a card, it is "five of a kind", and so on down to 2. We only have to do a bit more for two and three. In both cases we have to check whether there is actually another pair in the hand. If nothing matches, we have a "high card". 

In [8]:
example_hand._determine_hand_value().name

'TWO_PAIR'

Now, we overwrite the `__lt__` method for Hands as well. But here we have to be a bit more careful. If the hand types differ, we can just compare those and are done. If that is not the case, we iterate over both hands and compare the integer representations of those cards that we get from our `_map_card` method. As soon as they are not equal, we are done. Now we can also compare hands:

In [9]:
Hand("2KJ2K") < Hand("2KJ2A")

False

We can see the results for all cards in our test dataset.

In [10]:
rows = [ (Hand(hand),int(bet)) for hand, bet in string_rows]   
for r in sorted(rows, key=lambda x: x[0]):
    print(f"Hand: {r[0]}\tBet: {r[1]}")

Hand: 32T3K (PAIR)	Bet: 765
Hand: KTJJT (TWO_PAIR)	Bet: 220
Hand: KK677 (TWO_PAIR)	Bet: 28
Hand: T55J5 (THREE_OF_A_KIND)	Bet: 684
Hand: QQQJA (THREE_OF_A_KIND)	Bet: 483


Now we just need to calculate the winnings as described in the task and we are done with task 1.

In [11]:
def calculate_winnings(hands):
    winnings = 0
    rank = 1
    for hand, bet in sorted(hands):
        winnings += bet*rank
        rank+=1
    return winnings

In [12]:
calculate_winnings(rows)

6440

## Part2 

Here things get more tricky. By introducing jokers into the mix, we are not just changing the value of a card, but also the way hands are interpreted. 

In [13]:
class HandWithJokers(Hand):
    
    @staticmethod
    def _map_card(c):
        if c == "J":
            return -1
        else:
            return Hand._map_card(c)
    
    def _count_cards(self):
        # Count all cards
        counter = Counter(self.cards)
        jacks = counter.get("J",0)
        
        # Find the non-jack-card that occurs most often
        highest_group = None 
        for c, number in counter.items():
            if c != "J":
                if (highest_group is None # Is it the first element?
                        or number > highest_group[1] # is the new card more frequent
                        or (number == highest_group[1] and self._map_card(c) > self._map_card(highest_group[0]))): # is it equally frequent, but the new card has higher value? 
                    highest_group = (c,number)     

        # Turn the jacks into cards of the highest group
        if highest_group is not None:
            counter[highest_group[0]] = highest_group[1] + jacks
            counter["J"] = 0
        
        # We do not need the cards, just their multiplicities
        return counter


To accound for the first change, we can simply change the integer mapping that dictates the sorting of cards. We overwrite `_map_card` in such a way that it returns `-1` for Jokers (`"J"`).

In [14]:
HandWithJokers._map_card("J")

-1

For the changed interpretation of hands, we first count the number of Jacks. In the case of `"2KJ2K"`, that would be exactly one. 
Now we search for the non-jack-card that occurs most often in our hand. If there are multiple cards of equaly high frequency, we want the higher card value.
In our example, this would be a pair of kings. Therefore, we turn our Jack into a King (by adding the Jack-count to the King-count). We now have 3 Kings.

In [15]:
example_with_joker = HandWithJokers("2KJ2K")
example_with_joker._count_cards()

Counter({'K': 3, '2': 2, 'J': 0})

We can take another look at our dataset. Note that there are now two hands with "Four of a kind".

In [16]:
rows = [ (HandWithJokers(hand),int(bet)) for hand, bet in string_rows]   
for r in sorted(rows, key=lambda x: x[0]):
    print(f"Hand: {r[0]}\tBet: {r[1]}")

Hand: 32T3K (PAIR)	Bet: 765
Hand: KK677 (TWO_PAIR)	Bet: 28
Hand: T55J5 (FOUR_OF_A_KIND)	Bet: 684
Hand: QQQJA (FOUR_OF_A_KIND)	Bet: 483
Hand: KTJJT (FOUR_OF_A_KIND)	Bet: 220


The rest works as before and we can just get the results.

In [17]:
print(calculate_winnings(rows))

5905
